# Maryland Select LiDAR tiles

The project examines slope rasters derived from LiDAR to find RCH. Evidence of RCH is more often found in forested areas where ground has not been plowed or built on and so it may be useful to examine only liDAR tiles that contain forested areas.

## Inputs

### LiDAR tiles shapefile

This is from FEMA, projected to crs 26985:
Get LiDAR las files from https://coast.noaa.gov/dataviewer/ Data Access Viewer

Bulk download and information
https://chs.coast.noaa.gov/htdata/lidar2_z/geoid18/data/8496/

lidar_tiles_path = "/content/drive/MyDrive/crane_maryland/lidar_tile_polygons/2012_fema_md_index_26985.shp"

### Land use: Forested

This is from the state of Maryland, filtered for only forested areas:

land_use_path = "/content/drive/MyDrive/crane_maryland/land_use/Maryland_Land_Use_Land_Cover_-_Land_Use_Land_Cover_2010_forest.shp"

## Outputs

A shapefile to contain LiDAR tiles that are in forested areas:

lidar_tiles_forested_polys_fp = "/content/drive/MyDrive/crane_maryland/polys/2012_fema_md_forested.shp"

## Method

Examine each LiDAR tile polygon and see if it overlaps the Geopandas Geoseries of forested polygons.

```
if (land_use_geoseries.overlaps(pred_poly).any()==True) or (land_use_geoseries.contains(pred_poly).any()==True) or (land_use_geoseries.within(pred_poly).any()==True):
```

## Notes

Using geoseries.overlap results in not including some LiDAR tile polygons that are entirely inside of a forested area. The overlap method must not work for that. https://geopandas.readthedocs.io/en/latest/docs/reference/api/geopandas.GeoSeries.overlaps.html "Geometries overlaps if they have more than one but not all points in common." Adding .contains and .within as well.


In [ ]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 26.2 MB/s 
     |████████████████████████████████| 6.3 MB 50.7 MB/s 
     |████████████████████████████████| 15.4 MB 81 kB/s 


In [ ]:
import os
import geopandas as gpd

lidar_tiles_path = "/content/drive/MyDrive/crane_maryland/lidar_tile_polygons/2012_fema_md_index_26985.shp"
land_use_path = "/content/drive/MyDrive/crane_maryland/land_use/Maryland_Land_Use_Land_Cover_-_Land_Use_Land_Cover_2010_forest.shp"
lidar_tiles_forested_polys_fp = "/content/drive/MyDrive/crane_maryland/polys/2012_fema_md_forested.shp"

# Create an empty geopandas GeoDataFrame for duplicates
lidar_tiles_forested_polys_df = gpd.GeoDataFrame()
lidar_tiles_forested_polys_df['Index'] = None
lidar_tiles_forested_polys_df['Name'] = None
lidar_tiles_forested_polys_df['URL'] = None
lidar_tiles_forested_polys_df['geometry'] = None

lidar_tiles_forested_polys_df.crs = ('EPSG:26985')

if os.path.exists(lidar_tiles_path) and os.path.exists(land_use_path):
    lidar_tiles_polys = gpd.read_file(lidar_tiles_path)
    land_use_polys = gpd.read_file(land_use_path)
    print("crs", lidar_tiles_polys.crs,land_use_polys.crs )
    print(type(land_use_polys))
    print(list(land_use_polys))
    print("lidar_tiles_polys",lidar_tiles_polys.shape, "land_use_polys",land_use_polys.shape)
    land_use_geoseries = land_use_polys['geometry']
    print(type(land_use_geoseries))
    # crs for Maryland
    area_crs = 26985
    matched_tiles_polys = list()    
    lidar_tiles_polys.to_crs(area_crs)
    land_use_polys.to_crs(area_crs)

    for index, row in lidar_tiles_polys.iterrows():
       #print("row",row)

       pred_poly = row[3]
       #print("-----")
       #print("pred_poly",type(pred_poly), pred_poly)
       #print("shape",land_use_geoseries.shape)
       #print("++++++++++++")

       #print(land_use_geoseries.overlaps(pred_poly).any())
       if (land_use_geoseries.overlaps(pred_poly).any()==True) or (land_use_geoseries.contains(pred_poly).any()==True) or (land_use_geoseries.within(pred_poly).any()==True):

           print("match", row[0])  
           lidar_tiles_forested_polys_df = lidar_tiles_forested_polys_df.append(row, ignore_index=True)
       else:
           print("    no match", row[0])  
          

if not lidar_tiles_forested_polys_df.empty:
    lidar_tiles_forested_polys_df.to_file(lidar_tiles_forested_polys_fp)
    print("Total forested tiles:", len(lidar_tiles_forested_polys_df))
else:
    print("empty")

# Wget the .laz files

+ Open the shape file containing the polygons of forested lidar tiles in Maryland.

+ For each row, wget the file at the url.

In [ ]:
import os
import geopandas as gpd
import time

# import the wget module
from wget import download
#
# create a downloader class.
class downloader:
        
    # Create a downloadfile method
    # Accepting the url and the file storage location
    # Set the location to an empty string by default.

    def downloadFile(self, url, location=""):
         # Download file and with a custom progress bar
        download(url, out = location)

downloadObj = downloader()

lidar_tiles_forested_polys_fp = "/content/drive/MyDrive/crane_maryland/polys/2012_fema_md_forested.shp"

if os.path.exists(lidar_tiles_forested_polys_fp):
    lidar_tiles_forested_polys = gpd.read_file(lidar_tiles_forested_polys_fp)
    
    print("crs", lidar_tiles_forested_polys.crs)
    print(type(lidar_tiles_forested_polys))

    print("lidar_tiles_forested_polys",lidar_tiles_forested_polys.shape)
    # crs for Maryland
    area_crs = 26985
    lidar_tiles_forested_polys.to_crs(area_crs)

    for index, row in lidar_tiles_forested_polys.iterrows():
       #print("row",row)

       print(row[0],row[1],row[2])
       
       downloadObj.downloadFile(str(row[2]),"/content/drive/MyDrive/crane_maryland/laz/laz_fema_2012_forested")
       
       time.sleep(10)


crs epsg:26985
<class 'geopandas.geodataframe.GeoDataFrame'>
lidar_tiles_forested_polys (2154, 4)
0 BLK_6/20120203_17SPD8599.laz https://chs.coast.noaa.gov/htdata/lidar2_z/geoid18/data/8496/BLK_6/20120203_17SPD8599.laz

1 BLK_6/20120203_17SPD8598.laz https://chs.coast.noaa.gov/htdata/lidar2_z/geoid18/data/8496/BLK_6/20120203_17SPD8598.laz

2 BLK_6/20120203_17SPD8296.laz https://chs.coast.noaa.gov/htdata/lidar2_z/geoid18/data/8496/BLK_6/20120203_17SPD8296.laz

3 BLK_6/20120203_17SPD8596.laz https://chs.coast.noaa.gov/htdata/lidar2_z/geoid18/data/8496/BLK_6/20120203_17SPD8596.laz

4 BLK_6/20120203_17SPD8190.laz https://chs.coast.noaa.gov/htdata/lidar2_z/geoid18/data/8496/BLK_6/20120203_17SPD8190.laz

5 BLK_6/20120203_17SPD8295.laz https://chs.coast.noaa.gov/htdata/lidar2_z/geoid18/data/8496/BLK_6/20120203_17SPD8295.laz

6 BLK_6/20120203_17SPD8499.laz https://chs.coast.noaa.gov/htdata/lidar2_z/geoid18/data/8496/BLK_6/20120203_17SPD8499.laz

7 BLK_6/20120203_17SPD7987.laz https://chs.coast

In [ ]:
!wget --version

GNU Wget 1.19.4 built on linux-gnu.

-cares +digest -gpgme +https +ipv6 +iri +large-file -metalink +nls 
+ntlm +opie +psl +ssl/openssl 

Wgetrc: 
    /etc/wgetrc (system)
Locale: 
    /usr/share/locale 
Compile: 
    gcc -DHAVE_CONFIG_H -DSYSTEM_WGETRC="/etc/wgetrc" 
    -DLOCALEDIR="/usr/share/locale" -I. -I../../src -I../lib 
    -I../../lib -Wdate-time -D_FORTIFY_SOURCE=2 -DHAVE_LIBSSL -DNDEBUG 
    -g -O2 -fdebug-prefix-map=/build/wget-Xb5Z7Y/wget-1.19.4=. 
    -fstack-protector-strong -Wformat -Werror=format-security 
    -DNO_SSLv2 -D_FILE_OFFSET_BITS=64 -g -Wall 
Link: 
    gcc -DHAVE_LIBSSL -DNDEBUG -g -O2 
    -fdebug-prefix-map=/build/wget-Xb5Z7Y/wget-1.19.4=. 
    -fstack-protector-strong -Wformat -Werror=format-security 
    -DNO_SSLv2 -D_FILE_OFFSET_BITS=64 -g -Wall -Wl,-Bsymbolic-functions 
    -Wl,-z,relro -Wl,-z,now -lpcre -luuid -lidn2 -lssl -lcrypto -lpsl 
    ftp-opie.o openssl.o http-ntlm.o ../lib/libgnu.a 

Copyright (C) 2015 Free Software Foundation, Inc.
License 

In [ ]:
print(lidar_tiles_forested_polys_df)

In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=2ae66ac6cec93ee835bc77fe411e95fc86c8dac52b2b1fcc2a3c797ad60fd43e
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
